In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from efficientnet_models import create_efficientnetb7_model
import wandb
from wandb.keras import WandbCallback

2023-03-29 19:01:50.357851: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-29 19:01:50.359106: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-29 19:01:50.382294: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-29 19:01:50.382678: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-29 19:01:51.095060: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

# Loading Data Set
Three sets are created: training, validation, and test. 
- Labels are generated based on the folder structure. Class name must correspond to the subfolder name.
- Loading in batches, of size 32, to reduce memory usage.
- Label mode is set to categorical, which means that the labels are encoded as a categorical vector.

Bilinear interploation is set to default. This specify the method used in the resizing procedure. By default aspect ratio is not perserved, i.e., the ratio between image width and height.

One hot encoding is utilized when label mode is set to categorical.


The image load documentation is available [here](https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory) and an example is available [here](https://keras.io/api/data_loading/image/).

In [3]:
train_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/train",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
val_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/val",
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    batch_size=32,
    image_size=(224, 224)
    )
test_ds = keras.utils.image_dataset_from_directory(
    directory="dataset/test",
    labels='inferred',
    label_mode='categorical',
    shuffle=False,
    batch_size= 32,
    image_size=(224, 224)
    )

Found 297792 files belonging to 12 classes.
Found 37219 files belonging to 12 classes.
Found 37235 files belonging to 12 classes.



# EfficientNetB7 

In [4]:
# initialize wandb
wandb.init(project="EfficientNetB7", config={"learning_rate": 0.001, "epochs": 30, "verbose": 1,"name": "EfficientNetB7", "architecture": "EfficientNetB7"})

# configs
cfg = wandb.config

# create the model
model = create_efficientnetb7_model(len(train_ds.class_names))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gabri-torland (nubs). Use `wandb login --relogin` to force relogin


258076736/258076736 [==============================] - 5s 0us/step


In [5]:
# list of callbacks
callbacks = [
            WandbCallback(mode="min", monitor="val_loss", save_graph=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=0.00001),
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
          ]

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [6]:
model.fit(train_ds, epochs=cfg.epochs, batch_size=32, verbose=cfg.verbose, validation_data=val_ds, callbacks=callbacks) # train the model

Epoch 1/30


2023-03-29 19:03:03.996948: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [37219]
	 [[{{node Placeholder/_4}}]]
2023-03-29 19:03:03.997101: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [37219]
	 [[{{node Placeholder/_0}}]]
2023-03-29 19:03:04.029092: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2977

KeyboardInterrupt: 

In [ ]:
wandb.finish()

In [ ]:
from confusion_matrix import make_confusion_matrix, plot_confusion_matrix

# create model from pd file
model = tf.keras.models.load_model("./artifacts/model-royal-glade-3:v3")

loss, accuracy = model.evaluate(test_ds, verbose=cfg.verbose)
print("Test loss:", loss)
print("Test accuracy:", accuracy)